In [1]:
import sys
from pathlib import Path
import os

# Get the current working directory (where notebook is executed from)
current_dir = Path.cwd()

# Start from current directory and search upward for project root
# Project root should contain pyproject.toml (not in src/)
project_root = current_dir
max_levels = 10  # Safety limit

for _ in range(max_levels):
    
    # Check if this directory contains pyproject.toml
    if (project_root / "pyproject.toml").exists():
        # Verify it's the actual project root (not a subdirectory)
        # Project root should have pyproject.toml and src/ directory
        if (project_root / "src").exists() and (project_root / "pyproject.toml").exists():
            break
    if project_root == project_root.parent:
        # Reached filesystem root
        break
    project_root = project_root.parent
else:
    # Fallback: go up 3 levels from current directory if we're in src/adapters/ai_chat/
    if "src" in str(current_dir) and "adapters" in str(current_dir):
        project_root = current_dir.parent.parent.parent

# Add project root to Python path (must be absolute path)
project_root = project_root.resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Change working directory to project root
os.chdir(project_root)

print(f"Current directory (before): {current_dir}")
print(f"Project root: {project_root}")
print(f"Working directory (after): {os.getcwd()}")
print(f"pyproject.toml exists: {(project_root / 'pyproject.toml').exists()}")
print(f"src/ exists: {(project_root / 'src').exists()}")


Current directory (before): f:\Dev\interview-service\interview-service\src\adapters\ai_chat
Project root: F:\Dev\interview-service\interview-service
Working directory (after): F:\Dev\interview-service\interview-service
pyproject.toml exists: True
src/ exists: True


In [2]:
from openai import OpenAI
from src.adapters.ai_chat.ai_chat import AIChat
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
import asyncio
import dotenv   
import os
from config import MODEL_NAME, TOKEN_LIMIT

dotenv.load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# Вариант с доменом без порта (HTTPS):
BASE_URL = "https://llm.t1v.scibox.tech/v1"
# Альтернатива с IP:порт
# BASE_URL = "http://45.145.191.148:4000/v1"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
print(client)

In [3]:
print(MODEL_NAME, TOKEN_LIMIT)

qwen3-32b-awq 25000


In [4]:

# resp = client.chat.completions.create(
#     model="qwen3-32b-awq",
#     messages=[
#         {"role": "system", "content": "Ты дружелюбный помощник"},
#         {"role": "user", "content": "Расскажи анекдот"},
#     ],
#     temperature=0.7,
#     top_p=0.9,    max_tokens=20000,
# )

# print(resp.choices[0].message.content)

In [5]:
# with client.chat.completions.stream(
#     model="qwen3-32b-awq",
#     messages=[{"role": "user", "content": "Сделай краткое резюме книги Война и мир"}],
#     max_tokens=20000,
# ) as stream:
#     for event in stream:
#         if event.type == "chunk":
#             delta = getattr(event.chunk.choices[0].delta, "content", None)
#             if delta:
#                 print(delta, end="", flush=True)
#         elif event.type == "message.completed":
#             print()  # newlinefrom openai import OpenAI
# import dotenv   
# import os

# dotenv.load_dotenv()

# API_KEY = os.getenv("OPENAI_API_KEY")
# # Вариант с доменом без порта (HTTPS):
# BASE_URL = "https://llm.t1v.scibox.tech/v1"
# # Альтернатива с IP:порт
# # BASE_URL = "http://45.145.191.148:4000/v1"

# client = OpenAI(api_key=API_KEY, base_url=BASE_URL)


In [6]:
# from ai_utils.misc import get_chat_completion_stream

# stream = get_chat_completion_stream(client, "qwen3-32b-awq", [{"role": "user", "content": "Сделай задачу для собеседования AI разработчика на Python и тесты для нее"}], 20000)

# for chunk in stream:
#     print(chunk, end="", flush=True)
    


In [3]:
import asyncio
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
from datetime import timedelta

# Create example vacancy info
vacancy_info = VacancyInfo(
    profession="Python разработчик / Data Scientist",
    position="Junior Python Developer",
    requirements="Pandas, Numpy, Tensorflow, PyTorch, SQLAlchemy",
    questions="",
    tasks=None,
    task_ides=None,
    interview_plan="",  # Will be generated by create_chat
    duration=timedelta(minutes=30)
)

# Create example chat history
chat_history = [
    # Message(
    #     role=RoleEnum.USER,
    #     type=TypeEnum.QUESTION,
    #     content="Привет! Можете рассказать о вакансии?"
    # ),
    # Message(
    #     role=RoleEnum.AI,
    #     type=TypeEnum.ANSWER,
    #     content="Конечно! Мы ищем Senior Python Developer с опытом работы с FastAPI."
    # )
]

# Create example task
task = Task(
    type=TaskType.CODE,
    language="Python",
    description="Реализуйте функцию для валидации email адреса"
)


In [ ]:

# Test create_chat function
async def test_create_chat():
    ai_chat = AIChat()
    
    # create_chat returns updated VacancyInfo with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("=== Updated Vacancy Info ===")
    print(f"Profession: {updated_vacancy.profession}")
    print(f"Position: {updated_vacancy.position}")
    print(f"\nInterview Plan (first 300 chars):")
    print(updated_vacancy.interview_plan + "..." if len(updated_vacancy.interview_plan) > 300 else updated_vacancy.interview_plan)
    print(f"\nFull Interview Plan Length: {len(updated_vacancy.interview_plan)} characters")

# Test generate_welcome_message function
async def test_generate_welcome_message():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("\n=== Welcome Message (streaming) ===\n")
    welcome_chunks = []
    
    # ❌ before:
    # async for chunk in ai_chat.generate_welcome_message(updated_vacancy, chat_history):

    # ✅ after:
    stream = await ai_chat.generate_welcome_message(updated_vacancy, chat_history)
    async for chunk in stream:
        welcome_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Welcome Message ===")
    full_welcome = "".join(welcome_chunks)
    print(full_welcome)

# Run the async functions
print("Testing create_chat...")
await test_create_chat()

print("\n" + "="*50 + "\n")

print("Testing generate_welcome_message...")
await test_generate_welcome_message()


Testing create_chat...
=== Updated Vacancy Info ===
Profession: Python разработчик / Data Scientist
Position: Junior Python Developer

Interview Plan (first 300 chars):
INTERNAL INTERVIEW PLAN – DO NOT SHARE WITH CANDIDATE  

1. **Warm-up (10-15 min)**  
   - [theory] *Explain the key differences between Pandas DataFrames and NumPy arrays. When would you use one over the other?*  
   - [theory] *What is the role of SQLAlchemy in Python applications? How does it differ from raw SQL?*  

2. **Core Coding Tasks (40-50 min)**  
   - **Task 1: Data Manipulation [coding]**  
     *Write a script that loads a CSV file into a Pandas DataFrame, filters rows where a column 'value' exceeds 100, and calculates the mean of another column 'score' grouped by a categorical column 'category'.*  
   - **Task 2: Machine Learning Basics [coding]**  
     *Using either TensorFlow or PyTorch, create a simple neural network with one hidden layer to classify the Iris dataset (assume preloaded data). Output th

In [6]:
# Test create_response function (for reference)
async def test_create_response():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    # Then create response with task
    print("\n=== Response to Task (streaming) ===\n")
    response_chunks = []
    stream = await ai_chat.create_response(updated_vacancy, chat_history, task)
    async for chunk in stream:
        response_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Response ===")
    full_response = "".join(response_chunks)
    print(full_response)

# Run the async function
await test_create_response()



=== Response to Task (streaming) ===

Thinking...
Finished thinking.


Пожалуйста, начните с объяснения, как вы планируете валидировать email. Например:  
- Какие критерии вы будете проверять (например, наличие символа @, корректность домена)?  
- Будете ли вы использовать регулярные выражения или другой подход?  
- Какие граничные случаи вы считаете важными для проверки (например, email без точки в домене, с цифрами и т.д.)?  

После этого вы можете приступить к написанию кода.

=== Full Response ===
Thinking...
Finished thinking.


Пожалуйста, начните с объяснения, как вы планируете валидировать email. Например:  
- Какие критерии вы будете проверять (например, наличие символа @, корректность домена)?  
- Будете ли вы использовать регулярные выражения или другой подход?  
- Какие граничные случаи вы считаете важными для проверки (например, email без точки в домене, с цифрами и т.д.)?  

После этого вы можете приступить к написанию кода.


In [4]:
# Test stream_task function
async def test_stream_task():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    print(updated_vacancy.interview_plan)
    # Then stream the task
    print("\n=== Streaming Task Description ===\n")
    task_chunks = []
    
    stream = await ai_chat.stream_task(updated_vacancy, chat_history)
    async for chunk in stream:
        task_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Task Description ===")
    full_task = "".join(task_chunks)
    print(full_task)

# Run the async function
await test_stream_task()


INTERNAL INTERVIEW PLAN ONLY – DO NOT DISCLOSE TO CANDIDATE  

1. **Warm-up (5-7 min)**  
   - [theory] Explain the key differences between Pandas DataFrames and NumPy arrays. When would you use one over the other?  

2. **Theoretical Questions (10-12 min)**  
   - [theory] How does SQLAlchemy’s ORM abstract database interactions? Provide an example of a Python class mapping to a database table.  
   - [theory] Compare TensorFlow and PyTorch in terms of use cases and dynamic vs. static computation graphs.  

3. **Core Coding Tasks (25-30 min)**  
   - [coding] Write a NumPy script to normalize a 2D array (row-wise) and handle missing values (replace with column mean).  
   - [coding] Given a Pandas DataFrame with missing timestamps and categorical features, write code to:  
     - Fill missing timestamps with the previous valid value.  
     - One-hot encode categorical columns.  
   - [coding] Implement a simple neural network in PyTorch for binary classification (1 hidden layer, sigm

#TODO: Перевести на русский это в промпте